In [110]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [111]:
file_path = 'data_files/nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.91%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.01%,2.950200e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.01%,2.441740e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.69%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.71%,1.728250e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [112]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [113]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [114]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [115]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [54]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [55]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [56]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [57]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [97]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [59]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [13]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [60]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [61]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [62]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [63]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [64]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [65]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [21]:
page=10

In [131]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 100:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2015-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2018-08-01 00:00:00
258.80303955078125
2018-08-02 00:00:00
260.1318359375
1


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2020-09-18 00:00:00
17.14480972290039
2020-09-21 00:00:00
16.752452850341797
2


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

...

2019-03-20 00:00:00
49.369998931884766
2019-03-21 00:00:00
49.720001220703125
11


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2018-10-19 00:00:00
62.15605926513672
2018-10-22 00:00:00
61.97288513183594
12


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

IndexError: single positional indexer is out-of-bounds

In [84]:
data_dict

{'industry': 'Consumer Discretionary',
 'wr1': -0.011338714161929133,
 'wr2': 0.040786776473789525,
 'wr3': -0.04058799531456262,
 'wr4': -0.049109129017416805,
 'vol1': 10195800,
 'vol2': 10247900,
 'vol3': 11862300,
 'vol4': 12886300}

In [125]:
list_dict

[{'industry': 'Health Care',
  'wr1': -0.0034978174183757935,
  'wr2': 0.02028214911436432,
  'wr3': 0.03411657332750692,
  'wr4': 0.018146739363171644,
  'vol1': 5097700,
  'vol2': 4404600,
  'vol3': 2648800,
  'vol4': 4282300,
  'pe_ratio': 37.93,
  'debt_to_equity': 1.05,
  'quick_ratio': 1.11,
  'total_shareholder_return': 3.43,
  'profit_margin': 19.96,
  'free_cash_margin': 18.7,
  'volatility': 0.016771018882520496,
  'cpi': 251.712,
  'interest_rate': 2.4,
  'unemployment_rate': 4.0,
  'sma': 116.19000000000001,
  'rsi': 52.97148746135344,
  'ema': 116.56291666666667,
  'label': 1},
 {'industry': 'Consumer Discretionary',
  'wr1': -0.015111665307694633,
  'wr2': -0.008589622169587544,
  'wr3': -0.005562100424715203,
  'wr4': 0.032358802231248464,
  'vol1': 3996129,
  'vol2': 6545462,
  'vol3': 2852027,
  'vol4': 2170811,
  'pe_ratio': 48.23,
  'debt_to_equity': 3.89,
  'profit_margin': 7.53,
  'free_cash_margin': -0.73,
  'volatility': 0.01787906983522815,
  'cpi': 271.696,
  '

### Export data into csv

In [132]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data{page}.csv', index=False)


In [133]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,profit_margin,free_cash_margin,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label
0,Health Care,0.015844,0.033352,-0.001093,0.037370,395300.0,351100.0,233400.0,237500.0,100.09,...,15.27,27.71,0.036606,252.146,1.91,3.8,258.496667,65.806680,258.489167,1
1,Utilities,0.055619,0.013090,-0.018436,-0.006659,6893600.0,4346300.0,3703300.0,4413900.0,49.92,...,-3.74,2.75,0.031338,260.280,0.09,7.9,18.426667,91.773779,18.490833,0
2,Industrials,-0.172708,-0.050760,-0.020746,-0.006639,3345400.0,3724800.0,13467200.0,2709700.0,20.41,...,17.59,7.03,0.022142,252.885,2.20,3.8,56.080000,0.000000,56.650000,0
3,Industrials,-0.058972,0.001839,-0.006849,-0.038208,1113300.0,962500.0,1064200.0,1497200.0,10.99,...,11.25,7.70,0.028119,237.111,0.38,4.9,42.940000,28.151403,42.614583,1
4,Consumer Discretionary,-0.090843,-0.021510,0.133471,0.013261,10115300.0,9382000.0,9233800.0,8667700.0,206.67,...,1.60,-13.90,0.039476,238.316,0.15,5.1,119.533333,7.001333,117.786250,0
5,Consumer Discretionary,-0.017927,0.046935,-0.038725,-0.030748,833800.0,482400.0,320200.0,388600.0,32.20,...,16.09,17.14,0.019189,287.504,0.08,3.6,370.896667,49.564383,372.766250,0
6,Technology,0.035951,-0.046128,-0.089054,0.056359,192240.0,369775.0,453050.0,188071.0,19.92,...,15.59,-5.01,0.079959,296.311,1.58,3.6,182.183333,53.972257,180.360000,0
7,Consumer Discretionary,-0.023722,0.014667,-0.008646,0.000289,3169800.0,2961300.0,1809300.0,1713000.0,22.84,...,7.48,16.65,0.018298,251.233,2.20,3.9,138.778756,24.539282,137.636811,0
8,Industrials,0.031391,-0.000797,0.020342,0.037131,2086800.0,1950900.0,1716900.0,1525000.0,27.53,...,3.84,9.87,0.018435,252.006,1.91,3.8,38.480000,99.408284,38.520833,0
9,Utilities,0.039671,0.022661,-0.051646,-0.009612,4695500.0,4942000.0,6398200.0,5010600.0,18.73,...,4.50,-31.54,0.017750,301.836,4.58,3.5,28.663333,84.386617,28.719583,1


In [21]:
page = 1

In [95]:
print(ticker, input_date)

psa 2018-03-14 00:00:00


In [134]:
import pandas as pd
import glob

# Get a list of all CSV files in the current directory
csv_files = glob.glob('data*.csv')

# Initialize an empty DataFrame to store the concatenated data
concatenated_data = pd.DataFrame()

# Loop through each CSV file and concatenate its data to the DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    concatenated_data = pd.concat([concatenated_data, df])

# Write the concatenated data to a new CSV file
concatenated_data.to_csv('GJ_data.csv', index=False)

print("Concatenation completed. Data saved to 'concatenated_data.csv'")


Concatenation completed. Data saved to 'concatenated_data.csv'
